# Query Translation 
- Vấn đề ảnh hưởng tới hiệu quả của quá trình trích xuất thông tin đó là việc người dùng cung cấp thông tin mù mờ, không rõ ràng. Việc câu hỏi không rõ ràng dẫn tới trích xuất thông tin cũng không rõ ràng -> ảnh hưởng tới việc hiểu của LLM và kết quả của LLM 

- Một cách để giải quyết vấn đề này là thực hiện một vài phương pháp tiền xử lý đầu vào của người dùng (xử lý câu hỏi của người dùng) nhằm giúp câu hỏi trở nên rõ ràng, nhiều góc độ hơn. 

- Query Translation là một chuỗi các hành động xử lý trong quá trình embedding giữa query và docs để đảm bảo thông tin được trích xuất luôn được đảm bảo tính chính xác.   

- Các cách để chuyển đổi câu query : 

<p align="center">
    <img src="../doc/image/general-approaches-transform-question.png" alt="basic-pipeline" width="400"/>
</p>

# Setup 

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [4]:
# Indexing 
import bs4 
from langchain_community.document_loaders import WebBaseLoader

# loader thông qua nguồn là trang web
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Faker_(gamer)",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body-content")
        )
    ),
)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma 

# khởi tạo vectorstore lưu trữ thông tin 
vectorstore = Chroma.from_documents(documents= splits, 
        embedding= GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY))


retriever = vectorstore.as_retriever()

# Re-written 
- Với các câu hỏi khó hiểu do cách diễn đạt của người dùng (không trừu tượng quá mà cũng không quá cụ thể), có hai cách thức chính để xử lý với dạng câu hỏi kiểu này đó là Multi-query và RAG-Fusion 


## Multi-query 
1. **Ý tưởng**: Ta có thể giải quyết vấn đề trên thông qua việc chuyển đổi, viết lại câu query của người dùng theo nhiều các khác nhau. Việc này đựa trên niềm tin rằng ngôn từ mà người dùng xử dụng sau khi embedding không thể trích xuất ra từ ngữ một cách chính xác. Việc viết lại sẽ cho câu query nhiều góc độ quan sát hơn, thông tin được đa dạng hơn

<p align="center">
    <img src="../doc/image/multi-query.png" alt="basic-pipeline" width="400"/>
</p>

In [14]:
## cài đặt multi-query ## 

# viết lại tập câu hỏi thông qua mô hình ngôn ngữ lớn. 
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspective = ChatPromptTemplate.from_template(template)



generate_queries = (
    prompt_perspective
    | ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

generate_queries


"""
INPUT : 
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: How many World Championship titles has Faker won in his League of Legends career?

RENDER OUTPUT : 
Here are five alternative ways to phrase the question "How many World Championship titles has Faker won in his League of Legends career?" to enhance search results in a vector database:

1. What is the total number of League of Legends World Championships won by Faker?
2. How many times has Faker been crowned a League of Legends World Champion?
3. What is Faker's World Championship win count in professional League of Legends?
4. In the history of League of Legends, how many World Championship titles belong to Faker?
5.  List the years Faker won the League of Legends World Championship. 

"""

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relevant documents from a vector \ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search. \nProvide these alternative questions separated by newlines. Original question: {question}'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000205A8E8D2D0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x00000205A8F9A110>, default_metad

In [17]:
from langchain.load import dumps, loads

def get_unique_union(docs : list[list]): 
    # lấy ra thông tin duy nhất được trích xuất từ câu hỏi đầu vào 
    flattened_docs = [dumps(doc) for sublist in docs for doc in sublist]

    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]


question = "How many World Championship titles has Faker won in his League of Legends career?"

# tạo ra retrieval chain, với đầu vào là câu một số câu query được tạo ra truóc đó -> lấy doc liên quan thông qua retriever -> tìm docs duy nhất trong số đó. 
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


6

In [18]:
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Faker_(gamer)'}, page_content='During the 2023 LCK Spring Split, Faker achieved another LCK record. On January 20, 2023, in a match against KT Rolster, Faker surpassed Kang "Gorilla" Beom-hyeon to claim the record for the most career assists in the history of the LCK at 4,137.[77] On July 2, in the 2023 LCK Summer Split, Faker was sidelined due to an arm injury. His absence from competitive play extended for a duration of four weeks. During this time, T1\'s record fell from 6–2 to 7–9. Faker returned to the starting roster on August 2, helping the team defeat the Kwangdong Freecs.[78][79] Winning their last game of the season, T1 finished with a 9–9 record, securing the fifth seed in the LCK Summer playoffs.[80] T1 reached the LCK Summer finals, where they lost to Gen.G. Finishing with the most championship points in the LCK, the team qualified for the 2023 World Championship, marking Faker\'s eighth Worlds appearance.[81][82

## RAG-Fusion 
- Một nhược điểm của việc sử dụng multi-query đó là với số lượng query sinh ra sẽ có một số lượng lớn context được lựa chọn, làm cho input của mô hình LLM trở lên vô cùng lớn -> có thể khiến mô hình hoạt động kém hoặc không hoạt động. Bên cạnh đó, multi-query chỉ làm đa dạng context, nó có thể lấy ra những context không đúng hoặc ít liên quan tới câu hỏi. Để cải thiện vấn đề này, người ta dã thêm một module với nhiệm vụ là lọc bỏ những context không liên quan, và lựa chọn số lượng context giới hạn 


<p align="center">
    <img src="../doc/image/rag-fusion.png" alt="basic-pipeline" width="400"/>
</p>


In [8]:
## PROMPT ## 
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Generate multi-query from query 
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# chain sinh ra nhiều câu query 
generate_query = (
    prompt_rag_fusion
    | ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [14]:
## Tính rank của document ## 
from langchain.load import dumps, loads
def reciprocal_rank_fusion(results: list[list], k=60):
    """Hàm tính rank-fusion của các document, trong đó, nó nhận chuỗi các chuỗi document (được lấy ra từ mỗi query trước đó) 
        đã được đánh rank trước đó và tham số mịn k để cho phương trình RRF để tránh tính toán 1/0"""
    
    # khởi tạo fused_score dict dể lưu điểm của các document một cách duy nhất.
    fused_scores = {}

    # duyệt qua các list doc trong result 
    for docs in results:
        # Duyệt qua các doc trong list doc và rank của nó 
        for rank, doc in enumerate(docs):
            # chuyển đổi doc thành string và là key của fused_scores
            doc_str = dumps(doc)
            # nếu doc chưa tồn tại trong dictionary -> nó có score là 0 
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # trích xuất thông tin của scores trước đó 
            previous_score = fused_scores[doc_str]
            # Update score sử dụng RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Ranking lại toàn bộ result thông qua việc sort theo thứ tự giảm dần dựa vào fused_score
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return reranked_results

In [16]:
## Sinh query và trích xuất thông tinh đã được đánh rank ## 
question = "How many World Championship titles has Faker won in his League of Legends career?"
retrieval_chain_rag_fusion = generate_query | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

d:\WorkSpace_Trung\AI\simple-rag\rag\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


15

: 

In [ ]:
# final pipeline with multi-query 

from operator import itemgetter 
from langchain_core.runnables import RunnablePassthrough 

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template) 
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)

# xây dựng final chain thực hiện liên tục sinh multi-query -> lấy context phù hợp -> prompt -> llm -> ouput 
final_rag_chain = (
    {'context': retrieval_chain, 
     'question' : itemgetter("question")}
    | prompt 
    | llm 
    | StrOutputParser()
)


# trích xuất câu hỏi từ chain 
final_rag_chain.invoke({"question":question})

# Query Decomposition
- Với một câu query với lượng thông tin quá lớn, việc trích xuất thông tin có thể không chính xác, hoặc quá chung chung. Để xác nhận điều này, thay vì viết lại câu theo nhiều các khác nhau, người ta có thể chia nhỏ thông tin câu query thành nhiều phần và sử dụng phương pháp trích xuất song song hoặc đệ quy để trích xuất thông tin từ đó. 


In [7]:
# sinh các sub-question từ câu hỏi gốc thông qua prompt 
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# llm 
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)

# chain 
generate_queries_decomposition = (
    prompt_decomposition
    | llm 
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question": question})
questions

['Here are 3 search queries related to the components of an LLM-powered autonomous agent system:',
 '',
 '1. **"LLM agent architecture memory perception action loop"** (This query targets the core structural elements and how they interact)',
 '2. **"Tools and APIs for building autonomous agents with large language models"** (This focuses on the practical building blocks and available resources)',
 '3. **"Role of planning, reasoning, and learning in LLM-based autonomous agents"** (This query dives into the cognitive capabilities that LLMs bring to such systems) ',
 '']

## Recursive Answering Approach
- Với hướng tiếp cận sử dụng đệ quy, sau khi tách các câu hỏi, thông qua vectorstore, dữ liệu dần được trích xuất, và lần lượt đưa chúng qua mô hình ngôn ngữ lớn. Thông tin trả lời của câu sau sẽ được tách ra thành thông tin trả lời của câu trước đó. 

<p align="center">
    <img src="../doc/image/recur-decompo-query.png" alt="basic-pipeline" width="600"/>
</p>


In [8]:
# Prompt sinh câu trả lời thông qua việc lấy đệ quy 
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer): 
    ''' chuẩn hóa câu hỏi - câu trả lời '''

    format = f'Question: {question}\nAnswer: {answer}\n'
    return format.strip()


# llm 
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)
q_a_pairs = ""

for q in questions: 

    rag_chain = (
        {'context': itemgetter("question") | retriever, # trích xuất thông tin từ question  
         'question': itemgetter("question"), # câu hỏi được generate ở trên 
         'q_a_pairs': itemgetter("q_a_pairs")} # tạo cặp câu hỏi q-a-pair 
        | decomposition_prompt
        | llm 
        | StrOutputParser()
    )

    # trích xuất câu hỏi với từng cặp câu hỏi - trả lời trước đó. 
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})    
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n---\n" + q_a_pair

## Parallel Answering Approach
- Cách thức này chuyển đổi các câu hỏi - câu trả lời một cách song song, câu hỏi được sinh ra được mô hinh trả lời song song và sau đó được tổng hợp thành một câu hỏi chính

<p align="center">
    <img src="../doc/image/para-decompo-query.png" alt="basic-pipeline" width="600"/>
</p>


In [ ]:
# trả lời lần lượt các câu hỏi 
from langchain import hub 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda


prompt_rag = hub.pull('rlm/rag-prompt')


def retrieval_and_rag(question, prompt_rag, subquestion_generator_chain): 
    '''RAG trên từng tập câu hỏi con '''

    # trích xuất câu hỏi con 
    sub_questions = subquestion_generator_chain.invoke({'question': question})

    result = []

    # trích xuất câu trả lời từ tập câu hỏi 
    for sub_quest in sub_questions: 

        # trích xuất docs liên quan tới sub_quest 
        retrieved_docs = retriever.get_relevant_documents(sub_quest)

        ans = (prompt_rag | llm | StrOutputParser()).invoke({'context': retrieved_docs, 'question': sub_quest})

        result.append(ans)

    return result, sub_questions

# sinh câu hỏi - câu trả lời 
answers, questions = retrieval_and_rag(question, prompt_rag, generate_queries_decomposition)

In [10]:
# tổng hợp câu hỏi - câu trả lời 

def format_qa_pairs(questions, answers): 
    ''' chuyển đổi đoạn question và answer '''
    formatted_string = ""

    for i, (question, answer) in enumerate(zip(questions, answers), start = 1): 
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"

    return formatted_string

context = format_qa_pairs(questions, answers)

template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm 
    | StrOutputParser()
)


final_answer = final_rag_chain.invoke({"question": question, "context": context})




# Step-back 
- Đề giúp câu trả lời được sinh ra có tính logic cao hơn, mở rộng context cho một câu hỏi của người dùng, người ta đã sử dụng phương pháp step-back (làm câu hỏi trở lên trừu tượng, khái quát hóa hơn). Context có thể được lấy trực tiếp từ query người dùng kết hợp với context được lấy thông qua câu hỏi trừu tượng hơn giúp cho câu trả lời được sinh ra hiệu quả hơn. 

<p align="center">
    <img src="../doc/image/step-back-query.png" alt="basic-pipeline" width="600"/>
</p>


In [ ]:
# sinh câu hỏi step-back thông qua few-shot learning 
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# ví dụ 
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    }, 
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# prompt template cho ví dụ 
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"), 
        ("ai", "{output}")
    ]
)

# khởi tạo few-shot learning prompt 
fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, 
    examples=examples
)


# khởi tạo prompt 
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        fewshot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

# chain sinh step-back question 
step_back_generation_chain = (prompt | llm | StrOutputParser())

In [ ]:
# ví dụ về sinh step-back question 
question = "What is task decomposition for LLM agents?"
step_back_generation_chain.invoke({"question": question}) 

In [ ]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

# final prompt 
response_prompt  = ChatPromptTemplate.from_template(response_prompt_template)

# khởi tạo final-chain 
final_chain = (
    {   
        # trích xuất context thông thường thông qua question 
        "normal_context": RunnableLambda(lambda x: x['question']) | retriever, 
        "step_back_context": step_back_generation_chain | retriever, # trích xuất thông tin liên quan tới step back 
        "question": lambda x: x['question'], 
    }
    | response_prompt
    | llm 
    | StrOutputParser()
)


# chạy thử chain 
final_chain.invoke({"question": question})




# HyDE 
- Đôi khi việc tìm hay so sánh độ tương đồng giữa những câu hỏi của người dùng và context phù hợp có thể không chính xác. Đều này đến từ việc document được chunks thành các đoạn lớn hơn nhiều so với input đầu vào, ma trận của nó thường dense. Để khắc phục điều đó, phương pháp HyDE chuyển đổi các query của người dùng thành các passage giống với định dạng của docs và sử dụng nó để trích xuất thông tin hiểu quả hơn

<p align="center">
    <img src="../doc/image/hyde-transform.png" alt="basic-pipeline" width="600"/>
</p>


In [ ]:
# sử dụng prompt để sinh ra docs từ question (hypothetical doc)
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)


generate_hypo_docs_chain = (
    prompt_hyde 
    | llm 
    | StrOutputParser()
)

# test thử 
generate_hypo_docs_chain("What is the meaning of life?")

In [ ]:
# trích xuất docs liên quan tới template 
retrieval_hype_chain = generate_hypo_docs_chain | retriever
re_docs = retrieval_chain.invoke({"question": question})


# Rag 
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

hype_prompt = ChatPromptTemplate.from_template(template)

final_hype_chain = (
    prompt 
    | llm 
    | StrOutputParser()
)

final_hype_chain.invoke({"context": re_docs, "question": question})

# Tham khảo 
[Query Translation for RAG (Retrieval Augmented Generation)Applications](https://raghunaathan.medium.com/query-translation-for-rag-retrieval-augmented-generation-applications-46d74bff8f07)